In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from scipy.stats import pearsonr as pearson
from scipy.stats import spearmanr as spearman
from math import isnan
from collections import Counter

In [2]:
train_file = "noun_train.csv"
raw_train_file = pd.read_csv(train_file)
raw_train_file.columns = [c.replace('.', '_') for c in raw_train_file.columns]

devte_file = "noun_devtest.csv"
raw_devte_file = pd.read_csv(devte_file)
raw_devte_file.columns = [c.replace('.', '_') for c in raw_devte_file.columns]
print(len(raw_train_file), len(raw_devte_file))
raw_data_file = raw_train_file.append(raw_devte_file, ignore_index=True)

2985 2189


In [3]:
def extract_dataframe(data):
    '''
    Input: Pandas csv dataframe obtained from MTurk
    
    Output: Pandas dataframe levelled by (User x Sentenced_ID)
    '''
    data["dicts"] = data["Input_var_arrays"].map(lambda x: json.loads(x))
    global_list = []
    
    for row in data.itertuples():
        for idx, local_dict in enumerate(row.dicts):
            temp_dict = local_dict.copy()
            var_part = "Answer_noun_part" + str(idx + 1)
            var_part_c = "Answer_noun_part_certainty" + str(idx + 1)
            var_kind = "Answer_noun_class" + str(idx + 1)
            var_kind_c = "Answer_noun_class_certainty" + str(idx + 1)
            var_abs = "Answer_noun_abs" + str(idx + 1)
            var_abs_c = "Answer_noun_abs_certainty" + str(idx + 1)
            temp_dict['part'] = getattr(row, var_part)
            temp_dict['part_conf'] = getattr(row, var_part_c)
            temp_dict['kind'] = getattr(row, var_kind)
            temp_dict['kind_conf'] = getattr(row, var_kind_c)
            temp_dict['abs'] = getattr(row, var_abs)
            temp_dict['abs_conf'] = getattr(row, var_abs_c)
            temp_dict['worker_id'] = row.WorkerId
            temp_dict['hit_id'] = row.HITId
            temp_dict['status'] = row.AssignmentStatus
            global_list.append(temp_dict)
    
    return pd.DataFrame(global_list)

In [6]:
raw_data = extract_dataframe(raw_data_file)
raw_data = raw_data[raw_data['status']!='Rejected']
raw_data = raw_data.reset_index(drop=True)
raw_data['sent_noun'] = raw_data['sent_id'].map(lambda x : x) + "_" +\
                           raw_data['noun_token'].map(lambda x: str(x))
# Rearrange the columns
cols = ['hit_id', 'worker_id','sent_noun', 'noun', 'sent_id','noun_token','part','part_conf',
        'kind','kind_conf','abs','abs_conf']
data = raw_data[cols]

long_cols = ['Split', 'Annotator.ID','Sentence.ID', 'Noun.Token','Noun',
             'Is.Particular', 'Part.Confidence', 'Is.Kind','Kind.Confidence',
             'Is.Abstract','Abs.Confidence']

long_data = data.copy()
long_data = long_data.rename(columns={'worker_id':'Annotator.ID', 'sent_id':'Sentence.ID',
                                      'noun_token':'Noun.Token',
                                      'noun':'Noun', 'part':'Is.Particular', 
                                      'part_conf':'Part.Confidence', 'kind':'Is.Kind', 
                                      'kind_conf':'Kind.Confidence', 'abs':'Is.Abstract', 
                                      'abs_conf':'Abs.Confidence'})

long_data['Split'] = long_data['Sentence.ID'].str[6:11]
long_data['Split'] = long_data['Split'].map(lambda x: x.rstrip('.c'))

ann_hash = {}
annid = 0
for ann in set(long_data['Annotator.ID'].values):
    annid += 1
    ann_hash[ann] = annid
long_data['Annotator.ID'] = long_data['Annotator.ID'].map(ann_hash)
long_data = long_data[long_cols]
long_data.to_csv('noun_long_data.tsv', sep="\t", index=False)